In [13]:
from langchain_ollama import ChatOllama
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

from IPython.display import display, Markdown

In [14]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [15]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [16]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/Users/ismasadou/Documents/kuude/fulfulde-translator/.venv/lib/python3.13/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [18]:
query = "Please list all shirts with sun protection in a markdown table with a summary of the product description."


In [21]:
llm = ChatOllama(model="llama3.2:latest", temperature=0.5)

In [22]:
response = index.query(question=query, llm=llm)

In [23]:
display(Markdown(response))

| Product ID | Name | Sun Protection Summary |
| --- | --- | --- |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ coverage, blocks 98% UV rays, wrinkle-free, machine washable and dryable |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+, traditional fit, 100% polyester, front and back cape venting, two front bellows pockets |
| 87 | Women's Tropical Tee, Sleeveless | UPF 50+ rated, high-performance fabric provides SPF 50+ sun protection, machine wash and dry |
| 679 | Women's Tropical Plaid Shirt | UPF 50+, lightest hot-weather shirt with a flattering fit, keeps you cool and comfortable with perspiration-wicking fabric |

Note: I've summarized the product descriptions to focus on the key points related to sun protection. Let me know if you'd like me to add anything else!

Step through

In [24]:
from langchain.document_loaders import CSVLoader

file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [25]:
docs = loader.load()

In [26]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [27]:
from langchain_ollama.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [29]:
embed = embeddings.embed_query('Hi, my name is Isma')

In [30]:
print(len(embed))

1024


In [31]:
print(embed[:10])

[-0.0014894401, 0.005513572, -0.021584243, 0.0007067605, 0.01556979, -0.022926742, 0.0219562, 0.00812847, 0.0044451538, 0.002671582]


In [32]:
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [33]:
query = "Please list all shirts with sun protection."

In [34]:
found_docs = db.similarity_search_with_score(query)

In [35]:
list(found_docs)

[(Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 374}, page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available."),
  np.float64(0.7344831326404913)),
 (Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 709}, page_content=": 709\nname: Sunrise Tee\ndescription: Stay co

In [36]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [37]:
retriever = db.as_retriever()

In [38]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [39]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all shirts with sun protection in a markdown table with a summary of the product description.")

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_17660/95260663.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all shirts with sun protection in a markdown table with a summary of the product description.")


In [40]:
display(Markdown(response))

Here is the markdown table with the shirts that have sun protection:

| **Product Name** | **Summary of Description** |
| --- | --- |
| CozyPrint Sweater Fleece Pullover | Rugged sweater-knit exterior and soft brushed interior for exceptional warmth and comfort, with UPF 50+ sun protection. |
| Women's Stop Flies Hoodie | Repels ticks, mosquitoes, flies, and other biting insects using No Fly Zone Technology, while also providing UPF 40+ sun protection. |

Note: I've only included shirts that have explicit sun protection mentioned in the product description. If you'd like to include other products with sun protection as well, please let me know!

In [41]:
from langchain.chains import RetrievalQA

In [43]:
qa_stuff = RetrievalQA.from_chain_type(retriever=retriever, llm=llm, chain_type="stuff", verbose=True)

In [44]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [45]:
response = qa_stuff.run(query)

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_17660/3485144879.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


In [46]:
display(Markdown(response))

| ID | Name | Description |
| --- | --- | --- |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable shirt with UPF 50+ coverage, blocking 98% of sun's harmful UV rays. |
| 709 | Sunrise Tee | Lightweight, high-performance fabric wicks away moisture and dries quickly, providing built-in sun protection with a rating of UPF 50+. |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Traditional fit with relaxed chest, sleeve, and waist, made of 100% polyester with front and back cape venting for ventilation. |
| 87 | Women's Tropical Plaid Shirt | Flattering fit with slightly fitted design, UPF 50+ rated sun protection, and features like wicking fabric and smooth buttons. |

Note: I've only included shirts with explicit sun protection ratings (UPF 50+) in the table.

In [47]:
response = index.query(query, llm=llm)

In [48]:
display(Markdown(response))

| **Name** | **Description** |
| --- | --- |
| Men's Plaid Tropic Shirt, Short-Sleeve | Lightweight hot-weather shirt with UPF 50+ coverage, providing 98% blockage of sun's harmful UV rays. |
| Men's Tropical Plaid Short-Sleeve Shirt | Traditional fit with 100% polyester fabric, featuring front and back cape venting for ventilation. |
| Women's Tropical Plaid Shirt | Flattering fit with UPF 50+ rated sun protection, keeping you cool and comfortable by wicking perspiration away from your skin. |
| Women's Tropical Tee, Sleeveless | Five-star sleeveless button-up shirt with built-in SunSmart UPF 50+ rated sun protection, providing a flattering fit and smooth buttons. |

Note: The descriptions provided are based on the information given in the product context.

In [49]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])